### 데이터 받아오기

In [1]:
import json
import numpy as np
import random
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

### 데이터 분리하기

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [5]:
reviewText = []
for i in range(len(review_data)):
    reviewText.append(review_data[i]["reviewText"])

In [6]:
reviewText[9]

"An amazing work. Realizing extensive use of Biblical imagery and sentence structure, &#34;The Prophet&#34; by Khalil Gibran is a literary classic. Influencing the Free Love movement of the 1960's, Gibran's master work explores themes of love, longing and loss."

In [7]:
y_data = []
for i in range(len(review_data)):
    y_data.append(review_data[i]["overall"])

In [8]:
from collections import Counter
Counter(y_data)

Counter({1.0: 4239, 2.0: 5102, 3.0: 10177, 4.0: 21555, 5.0: 52111})

<h3>데이터 전처리</h3>

In [9]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [10]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i]).get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

       

In [11]:
reviewText=review_to_words(reviewText)

c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file c:\users\jangy\miniconda3\envs\bi_project\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [12]:
len(reviewText)

93184

### 데이터 셔플

In [13]:
# c = list(zip(pre_data, y_data))
# random.shuffle(c)

In [14]:
# a, b = zip(*c)

In [15]:
# reviewText=list(a)
# overall=np.array(list(b))

In [16]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
#vectorizer = TfidfVectorizer(ngram_range=(1,2))

# tfidf = vectorizer.fit_transform(reviewText)

### 가장 작은 평점으로 갯수 맞추기

In [17]:
from collections import Counter
length=sorted(Counter(y_data).most_common())[0][1]

In [18]:
# pre_data=[]
# overall_data=[]
# for j in range(len(reviewText)):
#     try:
#         if 50<=len(reviewText[j]) or len(reviewText[j])<=1500:
#             pre_data.append(reviewText[j])
#             overall_data.append(y_data[j])
#     except:
#         print(j)

In [20]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(y_data):
        if (count==length): break
        elif (i==value):
            text.append(reviewText[index])
            y.append(value)
            count=count+1

In [21]:
Counter(y)

Counter({1.0: 4239, 2.0: 4239, 3.0: 4239, 4.0: 4239, 5.0: 4239})

In [22]:
len(text)

21195

<h3>데이터 셔플 다시</h3>

In [23]:
c = list(zip(text, y_data))
random.shuffle(c)

In [24]:
a, b = zip(*c)

In [25]:
text_data=list(a)
overall=np.array(list(b))

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(text_data)

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall , test_size=0.33, random_state=100)

In [28]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((14200, 929377), (14200,), (6995, 929377), (6995,))

### 예측

#### Multiclass SVM

In [29]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

result = OneVsRestClassifier(LinearSVC(random_state=100)).fit(x_train, y_train)
result = result.predict(x_test)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(result ,y_test)

0.55067905646890636

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00       314
        2.0       0.00      0.00      0.00       412
        3.0       0.09      0.00      0.01       758
        4.0       0.23      0.06      0.09      1535
        5.0       0.57      0.95      0.71      3976

avg / total       0.38      0.55      0.43      6995



### LogisticRegression

In [32]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(x_train, y_train)
logreg.predict(x_test)==y_test

array([ True,  True, False, ..., False,  True, False], dtype=bool)

In [33]:
print(y_test[:10], logreg.predict(x_test)[:10])

[ 5.  5.  4.  4.  5.  5.  4.  5.  5.  2.] [ 5.  5.  5.  5.  5.  5.  5.  5.  5.  5.]


In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)


0.56840600428877774

In [35]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=5)
metrics.auc(fpr, tpr)

0.5

In [36]:
logreg.predict(x_test)[:10]

array([ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.])

In [37]:
result=logreg.predict(x_test)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00       314
        2.0       0.00      0.00      0.00       412
        3.0       0.00      0.00      0.00       758
        4.0       0.00      0.00      0.00      1535
        5.0       0.57      1.00      0.72      3976

avg / total       0.32      0.57      0.41      6995



c:\users\jangy\miniconda3\envs\bi_project\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### INPUT 

In [39]:
texts = ["""I enjoyed this book, as long as I kept in mind that the book was written for young adults.  
         The characters were believable and it was easy to identify with their pain.""",
        "I think the author gets lost in his own thoughts and trys to make storys of something that's not there.",
        """I was very excited to start reading this book and although I did like it, 
        I thought that a lot of the words to describe body parts was kind of high-schoolish.  
        Didn't care for the cliff hanging ending.""",
        """I don't know if you are familiar with FANFIC on the internet but that what this book reminded me of.  
        I love the series and thought I would like the books...not so much.""",
        """As I love western era romances, this one was wonderful.  An easy read and ended the way we always hope....happily!"""]
overall=[4,1,3,2,5]
vecs = vectorizer.transform(texts) 

In [40]:
print(overall,logreg.predict(vecs))

[4, 1, 3, 2, 5] [ 5.  5.  5.  5.  5.]
